In [9]:
import matplotlib.pyplot as plt

In [2]:
def range_so_far(data):
    amp = []
    for j in range(50):
        single_stock_data = data[j]

        base_range = np.max(single_stock_data[-AMP_WINDOW:]) -  np.min(single_stock_data[-AMP_WINDOW:])
        amp.append(base_range)
    return amp

In [78]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


SHORT_TERM = 4              # Short term average
LONG_TERM = 15              # Long term average
PRICE_RANGE = 7             # The period to calculate price difference
AMP_WINDOW = 75            # The period to get stock amplitude

CHANGE_HOLDING = 500            # Price to change holding
AMP_LO_THRESHOLD = 7.5       # The threshold when price movement is considered low
AMP_HI_THRESHOLD = 1
PRICE_CHANGE_THRESHOLD = 0.01
MSE_THRESHOLD_2 = 0.04
SLOPE_THRESHOLD_2 = 2   # The threshold when price movement is considered high
MSE_THRESHOLD_1 = 0.05       # The threshold to control price volatility
SLOPE_THRESHOLD_1 = 0.05       # The threshold to control LR slope

nInst = 50
currentPos = np.zeros(nInst)
yesterday_sign = np.zeros(nInst)
first_if_activation_flag = np.zeros([750-15, nInst])

increase_holding = np.zeros([750-15, nInst])
decrease_holding = np.zeros([750-15, nInst])

def getMyPosition(prcSoFar):
	
    global currentPos, yesterday_sign, first_if_activation_flag, increase_holding, decrease_holding

    currentPrices = prcSoFar[:,-1] # price of last day

    amp = range_so_far(prcSoFar)
	
    # Get long term and short term average prices
    for stock in range(50):
        single_stock_data = prcSoFar[stock]

        # Use short term and long term average to determine sign
        long_mean = single_stock_data[-LONG_TERM:].mean()
        short_mean = single_stock_data[-SHORT_TERM:].mean()
        today_sign = np.sign(short_mean - long_mean)

        # Use a price window to make decision
        n_day_diff = single_stock_data[-PRICE_RANGE] - single_stock_data[-1]
        n_day_range = np.max(single_stock_data[-PRICE_RANGE:]) - np.min(single_stock_data[-PRICE_RANGE:])
        # two_day_diff = single_stock_data.loc[day-3, 'closePrice'] - single_stock_data.loc[day-1, 'closePrice']

        # Calculate the MSE of price movement during the range
        n_day_gap = np.diff(single_stock_data[-PRICE_RANGE:])
        LR = LinearRegression(n_jobs=-1).fit(np.array(range(PRICE_RANGE-1)).reshape(-1, 1), n_day_gap.reshape(-1,1))
        n_day_mse = mean_squared_error(n_day_gap, LR.predict(np.array(range(PRICE_RANGE-1)).reshape(-1, 1)))

        
        
        if currentPos[stock] * n_day_range * np.sign(n_day_diff) > np.abs(currentPos[stock] * currentPrices[stock]) * PRICE_CHANGE_THRESHOLD and n_day_mse > np.abs(n_day_diff*MSE_THRESHOLD_2) or ((LR.coef_ * currentPos[stock] < 0)[0][0] and (np.abs(LR.coef_) > SLOPE_THRESHOLD_2)[0][0]):
            if currentPos[stock] < 0:
                first_if_activation_flag[prcSoFar.shape[1]-15, stock] = -1
            if currentPos[stock] > 0:
                first_if_activation_flag[prcSoFar.shape[1]-15, stock] = 1
            currentPos[stock] = 0
            

        elif np.abs(n_day_diff) <= amp[stock]/AMP_LO_THRESHOLD or (n_day_mse > np.abs(n_day_diff*MSE_THRESHOLD_1) and (np.abs(LR.coef_) < SLOPE_THRESHOLD_1)[0][0]):
            pass
            
        elif np.abs(n_day_diff) >= amp[stock]/AMP_HI_THRESHOLD:
            value = today_sign * CHANGE_HOLDING
            currentPos[stock] -= value//currentPrices[stock]

    
        else:
            value = today_sign * CHANGE_HOLDING
            currentPos[stock] += value//currentPrices[stock]

            if today_sign == -1:
                increase_holding[prcSoFar.shape[1]-15, stock] = -1
            if today_sign == 1:
                increase_holding[prcSoFar.shape[1]-15, stock] = 1

        yesterday_sign[stock] = today_sign
	
    return currentPos

In [79]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
# from main import getMyPosition as getPosition

nInst = 0
nt = 0
commRate = 0.0010
dlrPosLimit = 10000

def loadPrices(fn):
    global nt, nInst
    #df=pd.read_csv(fn, sep='\s+', names=cols, header=None, index_col=0)
    df=pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    nt, nInst = df.values.shape
    return (df.values).T

pricesFile="./data/prices.txt"
prcAll = loadPrices(pricesFile)
print ("Loaded %d instruments for %d days" % (nInst, nt))

currentPos = np.zeros(nInst)

def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_,nt) = prcHist.shape
    for t in range(15,750): 
        prcHistSoFar = prcHist[:,:t]
        newPosOrig = getMyPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:,-1] #prcHist[:,t-1]
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        clipPos = np.clip(newPosOrig, -posLimits, posLimits)
        newPos = np.array([np.trunc(x) for x in clipPos])
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        print ("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" % (t,value, todayPL, totDVolume, ret))

    pll = np.array(todayPLL)
    (plmu,plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)



(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd
print ("=====")
print ("mean(PL): %.1lf" % meanpl)
print ("return: %.5lf" % ret)
print ("StdDev(PL): %.2lf" % plstd)
print ("annSharpe(PL): %.2lf " % sharpe)
print ("totDvolume: %.0lf " % dvol)
print ("Score: %.2lf" % score)

Loaded 50 instruments for 750 days
Day 15 value: -14.37 todayPL: $-14.37 $-traded: 14370 return: -0.00100
Day 16 value: -17.13 todayPL: $-2.76 $-traded: 32478 return: -0.00053
Day 17 value: -25.97 todayPL: $-8.84 $-traded: 50278 return: -0.00052
Day 18 value: -49.85 todayPL: $-23.88 $-traded: 69013 return: -0.00072
Day 19 value: -29.19 todayPL: $20.66 $-traded: 86093 return: -0.00034
Day 20 value: -71.37 todayPL: $-42.18 $-traded: 103153 return: -0.00069
Day 21 value: -56.41 todayPL: $14.96 $-traded: 123740 return: -0.00046
Day 22 value: -56.73 todayPL: $-0.32 $-traded: 137655 return: -0.00041
Day 23 value: -70.94 todayPL: $-14.21 $-traded: 157586 return: -0.00045
Day 24 value: -47.91 todayPL: $23.02 $-traded: 176593 return: -0.00027
Day 25 value: -60.35 todayPL: $-12.44 $-traded: 199145 return: -0.00030
Day 26 value: -116.26 todayPL: $-55.90 $-traded: 217349 return: -0.00053
Day 27 value: -250.16 todayPL: $-133.90 $-traded: 236951 return: -0.00106
Day 28 value: -182.27 todayPL: $67.90

In [95]:
# import matplotlib.pyplot as plt

# # Assuming you have the following data:
# # prcAll is a 2D array containing price data for all stocks
# # first_if_activation_flag is a 2D array containing activation flags for the first condition for all stocks

# for i in range(nInst):
#     # Create a figure and axis for each stock
#     fig, ax = plt.subplots(figsize=(10, 6))
    
#     # Calculate standardized stock price movement
#     standardized_price_movement = (prcAll[i, 500:] - np.mean(prcAll[i, 500:])) / np.std(prcAll[i, 500:])
    
#     # Plot standardized stock price movement
#     ax.plot(range(500, 750), standardized_price_movement, label='Stock Price Movement')
    
#     # Create a boolean mask for the points where first_if_activation_flag[:, i] == 1
#     activation_mask_pos = first_if_activation_flag[-250:, i] == 1
#     activation_mask_neg = first_if_activation_flag[-250:, i] == -1
    
#     # Add vertical lines at the positions where activation occurs
#     activation_positions = np.where(activation_mask_pos)[0]
#     for position in activation_positions:
#         ax.axvline(x=position+500, color='green', linestyle='--', linewidth=0.5)
    
#     activation_positions = np.where(activation_mask_neg)[0]
#     for position in activation_positions:
#         ax.axvline(x=position+500, color='red', linestyle='--', linewidth=0.5)

#     ax.set_xlabel('Day')
#     ax.set_ylabel('Standardized Stock Price')
#     ax.legend()
#     ax.set_title(f'Stock {i+1} Price Movement and Activation')

#     plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have the following data:
# prcAll is a 2D array containing price data for all stocks
# first_if_activation_flag is a 2D array containing activation flags for the first condition for all stocks

for i in range(nInst):
    # Create a figure and axis for each stock
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Calculate standardized stock price movement
    standardized_price_movement = (prcAll[i, 500:] - np.mean(prcAll[i, 500:])) / np.std(prcAll[i, 500:])
    
    # Plot standardized stock price movement
    ax.plot(range(500, 750), standardized_price_movement, label='Stock Price Movement')
    
    # Create boolean masks for the points where activation occurs
    activation_mask_pos = increase_holding[-250:, i] == 1
    activation_mask_neg = increase_holding[-250:, i] == -1
    activation_mask_close_pos = first_if_activation_flag[-250:, i] == 1
    activation_mask_close_neg = first_if_activation_flag[-250:, i] == -1
    
    # Add vertical lines at the positions where activation occurs
    activation_positions = np.where(activation_mask_pos)[0]
    for position in activation_positions:
        ax.axvline(x=position+500, color='green', linestyle='--', linewidth=0.5)
    
    activation_positions = np.where(activation_mask_neg)[0]
    for position in activation_positions:
        ax.axvline(x=position+500, color='red', linestyle='--', linewidth=0.5)

    activation_positions = np.where(activation_mask_close_pos)[0]
    ax.scatter(
        x=activation_positions+500,
        y=standardized_price_movement[activation_positions],
        color='red',
        marker='o',
        s = 50
    )

    activation_positions = np.where(activation_mask_close_neg)[0]
    ax.scatter(
        x=activation_positions+500,
        y=standardized_price_movement[activation_positions],
        color='green',
        marker='o',
        s = 50
    )

    ax.set_xlabel('Day')
    ax.set_ylabel('Standardized Stock Price')
    ax.legend()
    ax.set_title(f'Stock {i+1} Price Movement and Activation')

    plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have the following data:
# prcAll is a 2D array containing price data for all stocks
# first_if_activation_flag is a 2D array containing activation flags for the first condition for all stocks

for i in range(nInst):
    # Create a figure and axis for each stock
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Calculate standardized stock price movement
    standardized_price_movement = (prcAll[i, 500:] - np.mean(prcAll[i, 500:])) / np.std(prcAll[i, 500:])
    
    # Plot standardized stock price movement
    ax.plot(range(500, 750), standardized_price_movement, label='Stock Price Movement')
    
    # Create boolean masks for the points where activation occurs
    activation_mask_pos = increase_holding[-250:, i] == 1
    activation_mask_neg = increase_holding[-250:, i] == -1
    
    # Add vertical lines at the positions where activation occurs
    activation_positions = np.where(activation_mask_pos)[0]
    for position in activation_positions:
        ax.axvline(x=position+500, color='green', linestyle='--', linewidth=0.5)
    
    activation_positions = np.where(activation_mask_neg)[0]
    for position in activation_positions:
        ax.axvline(x=position+500, color='red', linestyle='--', linewidth=0.5)

    ax.set_xlabel('Day')
    ax.set_ylabel('Standardized Stock Price')
    ax.legend()
    ax.set_title(f'Stock {i+1} Price Movement and Activation')

    plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have the following data:
# prcAll is a 2D array containing price data for all stocks
# first_if_activation_flag is a 2D array containing activation flags for the first condition for all stocks

for i in range(nInst):
    # Create a figure and axis for each stock
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Calculate standardized stock price movement
    standardized_price_movement = (prcAll[i, 500:] - np.mean(prcAll[i, 500:])) / np.std(prcAll[i, 500:])
    
    # Plot standardized stock price movement
    ax.plot(range(500, 750), standardized_price_movement, label='Stock Price Movement')
    
    # Create boolean masks for the points where activation occurs
    activation_mask_pos = increase_holding[-250:, i] == 1
    activation_mask_neg = increase_holding[-250:, i] == -1
    
    # Add vertical lines at the positions where activation occurs
    activation_positions = np.where(activation_mask_pos)[0]
    for position in activation_positions:
        ax.axvline(x=position+500, color='green', linestyle='--', linewidth=0.5)
    
    activation_positions = np.where(activation_mask_neg)[0]
    for position in activation_positions:
        ax.axvline(x=position+500, color='red', linestyle='--', linewidth=0.5)

    ax.set_xlabel('Day')
    ax.set_ylabel('Standardized Stock Price')
    ax.legend()
    ax.set_title(f'Stock {i+1} Price Movement and Activation')

    plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have the following data:
# prcAll is a 2D array containing price data for all stocks
# first_if_activation_flag is a 2D array containing activation flags for the first condition for all stocks

for i in range(nInst):
    # Create a figure and axis for each stock
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Calculate standardized stock price movement
    standardized_price_movement = (prcAll[i, 500:] - np.mean(prcAll[i, 500:])) / np.std(prcAll[i, 500:])
    
    # Plot standardized stock price movement
    ax.plot(range(500, 750), standardized_price_movement, label='Stock Price Movement')
    
    # Create boolean masks for the points where activation occurs
    activation_mask_close_pos = first_if_activation_flag[-250:, i] == 1
    activation_mask_close_neg = first_if_activation_flag[-250:, i] == -1

    activation_positions = np.where(activation_mask_close_pos)[0]
    ax.scatter(
        x=activation_positions+500,
        y=standardized_price_movement[activation_positions],
        color='red',
        marker='o',
        s = 50
    )

    activation_positions = np.where(activation_mask_close_neg)[0]
    ax.scatter(
        x=activation_positions+500,
        y=standardized_price_movement[activation_positions],
        color='green',
        marker='o',
        s = 50
    )

    ax.set_xlabel('Day')
    ax.set_ylabel('Standardized Stock Price')
    ax.legend()
    ax.set_title(f'Stock {i+1} Price Movement and Activation')

    plt.show()
